In [251]:
# Import pandas library as pd
import pandas as pd
import hashlib
import pathlib

In [252]:
# Save file name in a variable so that it can be re used
source_file_name = 'HR_Data_20200410.dat'

In [253]:
def genrate_primary_key_per_batch(file_path):
    """
    Generate md5 hash key for every batch and use this key to join header, detail, trailer tables
    :param file_path: yq remittance file path on the lambda server
    :return: batch_pk: md5 hash as string
    """
    batch_pk = hashlib.md5(pathlib.Path(file_path).read_bytes()).hexdigest()
    return batch_pk

In [254]:
# Invoke the batch key generation function
batch_key = genrate_primary_key_per_batch(file_path=source_file_name)

In [255]:
# Show batch key
batch_key

'2d20553c798d5e2d38a4b4bb5ef18c2f'

In [256]:
# Declare herader columns & widths based on file structure 
header_columns = ['RECID', 'FILENAME', 'CREATEDTIME']
header_widths = [1, 18, 14]

In [257]:
# Read HR fixed width text file using pandas library read_fwf method
header_df = pd.read_fwf(source_file_name, widths=header_widths, header=None, names=header_columns, nrows=1)

In [258]:
# Apply batch key to header dataframe
header_df['PK_BATCH'] = batch_key

In [259]:
# Check data types
header_df.dtypes

RECID          object
FILENAME       object
CREATEDTIME     int64
PK_BATCH       object
dtype: object

In [260]:
# convert intger to date time
header_df['CREATEDTIME'] = pd.to_datetime(header_df['CREATEDTIME'], format='%Y%m%d%H%M%S')

In [261]:
# Check data types after converting and see the converted data type
header_df.dtypes

RECID                  object
FILENAME               object
CREATEDTIME    datetime64[ns]
PK_BATCH               object
dtype: object

In [262]:
# show content of a pandas dataframe after converting to proper data types
header_df

,RECID,FILENAME,CREATEDTIME,PK_BATCH
0,H,HR-DATA-2020-04-10,2020-04-10 08:29:30,2d20553c798d5e2d38a4b4bb5ef18c2f


In [263]:
# Declare dept herader columns & widths based on file structure 
dept_columns = ['RECID', 'DEPTNO', 'DNAME', 'LOC']
dept_widths = [2, 2, 25, 15]

In [264]:
# Read HR fixed width text file using pandas library read_fwf method
dept_df = pd.read_fwf(source_file_name, widths=dept_widths, header=None, names=dept_columns)

In [265]:
# Filter dataframe for department records
dept_df = dept_df[dept_df['RECID'] == 'DR']

In [266]:
# Apply batch key to dept dataframe (FK stands for Foreign key)
dept_df['FK_BATCH'] = batch_key

In [267]:
# check data types
dept_df.dtypes

RECID       object
DEPTNO      object
DNAME       object
LOC         object
FK_BATCH    object
dtype: object

In [268]:
# Convert deptno column data type to int
dept_df['DEPTNO'] = dept_df['DEPTNO'].astype(int)

In [269]:
# Check data types after converting and see the converted data type
dept_df.dtypes

RECID       object
DEPTNO       int64
DNAME       object
LOC         object
FK_BATCH    object
dtype: object

In [270]:
# show content of a pandas dataframe after converting to proper data types
dept_df

,RECID,DEPTNO,DNAME,LOC,FK_BATCH
1,DR,10,ACCOUNTING,NEW YORK,2d20553c798d5e2d38a4b4bb5ef18c2f
2,DR,20,RESEARCH,DALLAS,2d20553c798d5e2d38a4b4bb5ef18c2f
3,DR,30,SALES,CHICAGO,2d20553c798d5e2d38a4b4bb5ef18c2f
4,DR,40,OPERATIONS,BOSTON,2d20553c798d5e2d38a4b4bb5ef18c2f


In [271]:
# Declare emp herader columns & widths based on file structure 
emp_columns = ['RECID', 'EMPNO', 'ENAME', 'JOB', 'MGR', 'HIREDATE', 'SAL', 'COMM', 'DEPTNO']
emp_widths = [2, 5, 25, 15, 5, 10, 6, 4, 2]

In [272]:
# Read HR fixed width text file using pandas library read_fwf method
emp_df = pd.read_fwf(source_file_name, widths=emp_widths, header=None, names=emp_columns)

In [273]:
# Filter dataframe for employee records
emp_df = emp_df[emp_df['RECID'] == 'ER']

In [274]:
# Apply batch key to emp dataframe (FK stands for Foreign key)
emp_df['FK_BATCH'] = batch_key

In [275]:
emp_df.dtypes

RECID        object
EMPNO        object
ENAME        object
JOB          object
MGR         float64
HIREDATE     object
SAL         float64
COMM        float64
DEPTNO      float64
FK_BATCH     object
dtype: object

In [276]:
# Convert hiredate column from string to date
emp_df['HIREDATE'] = pd.to_datetime(emp_df['HIREDATE'])

In [279]:
# Convert empno, mgr, deptno columns from string to int
emp_columns_dtypes = {'EMPNO': 'int', 'MGR': 'Int64', 'DEPTNO': 'int'}

In [280]:
# Apply the conversion using astype
emp_df = emp_df.astype(emp_columns_dtypes)

In [281]:
# Check data types after converting and see the converted data type
emp_df.dtypes

RECID               object
EMPNO                int64
ENAME               object
JOB                 object
MGR                  Int64
HIREDATE    datetime64[ns]
SAL                float64
COMM               float64
DEPTNO               int64
FK_BATCH            object
dtype: object

In [282]:
# show content of a pandas dataframe after converting to proper data types
emp_df

,RECID,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,FK_BATCH
5,ER,7369,SMITH,CLERK,7902,1980-12-17,800.0,0.1,20,2d20553c798d5e2d38a4b4bb5ef18c2f
6,ER,7499,ALLEN,SALESMAN,7698,1981-02-20,1600.0,300.0,30,2d20553c798d5e2d38a4b4bb5ef18c2f
7,ER,7521,WARD,SALESMAN,7698,1981-02-22,1250.0,500.0,30,2d20553c798d5e2d38a4b4bb5ef18c2f
8,ER,7566,JONES,MANAGER,7839,1981-02-04,2975.0,NaN,20,2d20553c798d5e2d38a4b4bb5ef18c2f
9,ER,7654,MARTIN,SALESMAN,7698,1981-09-28,1250.0,1400.0,30,2d20553c798d5e2d38a4b4bb5ef18c2f
10,ER,7698,BLAKE,MANAGER,7839,1981-01-05,2850.0,NaN,30,2d20553c798d5e2d38a4b4bb5ef18c2f
11,ER,7782,CLARK,MANAGER,7839,1981-09-06,2450.0,NaN,10,2d20553c798d5e2d38a4b4bb5ef18c2f
12,ER,7788,SCOTT,ANALYST,7566,1982-09-12,3000.0,NaN,20,2d20553c798d5e2d38a4b4bb5ef18c2f
13,ER,7839,KING,PRESIDENT,<NA>,1981-11-17,5000.0,NaN,10,2d20553c798d5e2d38a4b4bb5ef18c2f
14,ER,7844,TURNER,SALESMAN,7698,1981-08-09,1500.0,NaN,30,2d20553c798d5e2d38a4b4bb5ef18c2f


In [283]:
# Declare trailer columns & widths based on file structure 
trailer_columns = ['RECID', 'ROWCOUNT']
trailer_widths = [1, 5]

In [284]:
# Read HR fixed width text file using pandas library read_fwf method
trailer_df = pd.read_fwf(source_file_name, widths=trailer_widths, header=None, names=trailer_columns).tail(1)

In [285]:
# Apply batch key to trailer dataframe (FK stands for Foreign key)
trailer_df['FK_BATCH'] = batch_key

In [286]:
# Check data types
trailer_df.dtypes

RECID       object
ROWCOUNT    object
FK_BATCH    object
dtype: object

In [287]:
# Convert rowcount column from string to int
trailer_dtypes = {'ROWCOUNT': int}

In [289]:
# Apply the conversion using astype
trailer_df = trailer_df.astype(trailer_dtypes)

In [290]:
# show content of a pandas dataframe after converting to proper data types
trailer_df

,RECID,ROWCOUNT,FK_BATCH
19,T,18,2d20553c798d5e2d38a4b4bb5ef18c2f
